In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
data_dir = 'data'
lb = 'iwgsc_10_12_18/'
output_fasta = data_dir + '/res/sRNA_DEG.fasta'

In [2]:
results_file = data_dir + '/res/' + lb + '/Results.txt'
df_res = pd.read_csv(results_file,sep="\t")
df_res = df_res[df_res.DicerCall.isin(['20','21','22'])]
print('Res len:',len(df_res.index))

Res len: 1626


In [3]:
deg_file = data_dir + '/res/diffexpr-results.0.5.csv'
df_deg = pd.read_csv(deg_file,sep=",",comment='#')
print('DEG len:',len(df_deg.index))

DEG len: 14769


In [4]:
up = len(df_deg[df_deg.log2FoldChange > 0].index)
down = len(df_deg[df_deg.log2FoldChange < 0].index)
(up, down)

(10045, 4724)

In [60]:
df_deg_res = pd.merge(df_deg, df_res,left_on='Gene', right_on='Name' , how='left')
#only some cols
#df_deg_res = df_deg_res['#Locus','Name','MajorRNA','log2FoldChange']
df_deg_res.to_csv(data_dir + '/res/deg_data.csv', sep='\t', index=None)

In [ ]:
#save fasta
buffer_seqs = []
for k,v in df_deg_res.iterrows():
    if pd.isna(v.Name):
        new_id = v.Gene
        new_desc = 'log2fc:%f size:%i' % (v.log2FoldChange, len(v.Gene))
        seq = v.Gene
        record = SeqRecord(Seq(seq), id=new_id, description=new_desc)
        buffer_seqs.append(record)
    else:
        new_id = v.Name
        new_desc = 'chr:%s log2fc:%f size:%i' % (v['#Locus'],  v.log2FoldChange, len(v.MajorRNA))
        seq = v.MajorRNA
        record = SeqRecord(Seq(seq), id=new_id, description=new_desc)
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, output_fasta, "fasta")

In [41]:
df_res = pd.merge(df_deg, df_blast[['qseqid','sseqid']], left_on='Gene', right_on='qseqid', how='left')
df_res = df_res.drop(['qseqid'], axis=1)
df_res.rename(columns={'sseqid': 'miRBASE'}, inplace=True)
df_res = df_res.replace(np.nan, '', regex=True)
print(len(df_res.index))
df_res.head()

NameError: name 'df_blast' is not defined

In [6]:
#count DEG len for downregulated
df_down = df_deg[df_deg.log2FoldChange < 0]
y21 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(159, 37, 5, 1974)

In [20]:
#count DEG len for upregulated
df_up = df_deg[df_deg.log2FoldChange > 0]
y21 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(39, 4, 1, 13)

In [4]:
#count DEG len
y21 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(263, 45, 9, 2320)